# Building deep learning neural networks using TensorFlow layers 
([link](https://www.oreilly.com/ideas/building-deep-learning-neural-networks-using-tensorflow-layers?utm_content=buffer00205&utm_medium=social&utm_source=facebook.com&utm_campaign=buffer))

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('mnist_data', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting mnist_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting mnist_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting mnist_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting mnist_data/t10k-labels-idx1-ubyte.gz


https://www.tensorflow.org/versions/r1.1/get_started/mnist/beginners